<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/extract_images_tumorAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
patient_list_path = r"F:\SlitPhoto\20100101~20230130_スリット(写真室)\eyelid_conj_tumor_list.csv"
IDMap_path = r"F:\SlitPhoto\20100101~20230130_スリット(写真室)\combined_IDMap.csv"

In [ ]:
import pandas as pd

# Load the first CSV file
df1 = pd.read_csv(patient_list_path, encoding='cp932')

# Load the second CSV file
df2 = pd.read_csv(IDMap_path, encoding='cp932', header = None)

# Displaying the first few rows of each DataFrame to understand their structure
df1.head(), df2.head()


In [ ]:
df1

In [ ]:
df2

,0,1,2
0,555461,ZZZ,e34ed179965d41628e272b81a57c6ab8f98799dae2a107...
1,614306,ZZZ,19bfe243e355bc25bd03a5d4d5999bf70c7e6b597041cf...
2,619856,ZZZ,04ef158432ca0922834cf271bef41586602ec7e91579b9...
3,645895,ZZZ,6064196e1be80b3500111333c5b8a915d1f96e33fb83c2...
4,654439,ZZZ,08af16758d765b933fa5c1a5e94099fbe2b532e71ff743...
...,...,...,...
19895,29580727,ZZZ,c51ed7d508fff009e6dc198ee9ac2357165a850b658cf5...
19896,29600914,ZZZ,12cbdf26d8734d92acea4dd6e137124e529eb69940c1c0...
19897,29660120,ZZZ,3167c58674d3edb65467fbc688da4bf8754c922106d0d8...
19898,29950208,ZZZ,d88bfdbac7a42e0f14931620bdbda1c4edfeb0d5e98a82...


In [ ]:
# df2[df2[0]==11845831].loc[0,2]
df2[df2[0]==5815065].loc[:,2]

Series([], Name: 2, dtype: object)

In [ ]:
for id in df1["ID"]:
    print(id)
    hash = df2[df2[0]==int(id)].loc[:, 2].item()
    print(hash)

In [ ]:
for user_id in df1["ID"]:
    #print(user_id)
    try:
        hash_value = df2[df2[0] == int(user_id)].iloc[0, 2]
        print(hash_value)
    except:
        #print(f"No hash found for ID {user_id}")
        pass


In [ ]:
# Initialize a counter
hash_value_count = 0

for user_id in df1["ID"]:
    try:
        hash_value = df2[df2[0] == int(user_id)].iloc[0, 2]
        print(hash_value)
        hash_value_count += 1  # Increment the counter
    except:
        # If an exception occurs, the counter is not incremented
        pass

# Print the total count of hash_values printed
print("Total hash values printed:", hash_value_count)


...
Total hash values printed: 440
